In [1]:
from datasets import load_dataset

import torch

/home/karvsmech/miniconda3/envs/ptorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
print(f'features of the dataset: {dataset["train"].column_names}')

features of the dataset: ['article', 'highlights', 'id']


In [4]:
sample = dataset['train'][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample['article'])}
""")
print(sample['article'][:500])
print(f'\nSummary total length: {len(sample['highlights'])}')
print(sample['highlights'])


Article (excerpt of 500 characters, total length: 4051

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary total length: 281
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [5]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

sample_text = dataset['train'][1]['article'][:2000]
summaries = {}


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


summaries['baseline'] = three_sentence_summary(sample_text)
summaries

[nltk_data] Downloading package punkt to /home/karvsmech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/karvsmech/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'}

In [6]:
# GPT-2

from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline('text-generation', model='gpt2-xl', device=device)
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [7]:
# T5 model

pipe = pipeline('summarization', model='t5-large', device=device)
pipe_out = pipe(sample_text)
summaries['t5'] = "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

In [8]:
# BART

pipe = pipeline('summarization', model='facebook/bart-large-cnn', device=device)
pipe_out = pipe(sample_text)
summaries['bart'] = "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

In [9]:
# PEGASUS

pipe = pipeline('summarization', model='google/pegasus-cnn_dailymail', device=device)
pipe_out = pipe(sample_text)
summaries['pegasus'] = pipe_out[0]['summary_text'].replace("<n>", ".\n")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'gpt2': 'The mentally ill aren\'t being properly treated and are more likely to be arrested than not if you look at the statistics on the ninth floor of the pretrial detention center in Miami-Dade County.\nThe ninth floor is called the "forgotten floor."\nMany mentally ill inmates are housed there until charges are resolved.\nIn other words, they are there until they are locked up for good',
 't5': 'mentally ill inmates are housed on the ninth floor of a florida jail .\nmost face drug charges or charges

In [11]:
# comparing summaries from different models:

In [12]:
print('GROUND TRUTH')
print(dataset['train'][1]['highlights'])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

GPT2
The mentally ill aren't being properly treated and are more likely to be arrested than not if you look at the statistics on the ninth floor of the pretrial detention center in Miami-Dade County.
The ninth floor is call